# Abid - Phase 2
#### 12/9/2022

#### Relevant Libraries

In [10]:
# !pip install scikit-learn

In [11]:
#### Add Relevant Libraries
#################################################
from datetime import datetime, time, date, timedelta
import pandas as pd
import numpy as np
import sys
import os
CWD = os.getcwd()
WD = CWD.split('Main')[0] + 'Main/'
DFNC = WD + 'Functions'
sys.path.insert(1, DFNC)

#### Add External Functions
#################################################
from Scripts.FN_Support import Drct, Periods
from Scripts.CREATE_MODEL_REGRESSION import create_model 
from Scripts.RUN_PRODUCTION_REGRESSION import run_production_predict 

### Updated Patterns
- Below is an updated summery of patterns
- Several of these patterns are new from what I sent you last time
- All of them have a different column naming structure (more on that below...)

In [12]:
### Set the AGG Period Type
#######################################
AGG='5T'

### Load Pattern Summeries for ALL_CODES
#######################################
SUMMERY_FILE = f'{WD}Sources/aggs/{AGG}/Results/Pattern_Summeries/All.csv'
summeries = Drct.csv_from(SUMMERY_FILE)
ALL_CODES = list(summeries["pcode"].unique())
print(summeries[['pcode','filled']])


   pcode  filled
0   3BPD   17777
1   3BPU   18429
2   BFLD   13953
3   BFLU   14734
4    BOD   42491
5    BOU   73240
6   BTRD    2965
7   BTRU    2990
8    CCB   26940
9    CCS   26629
10    DB   69421
11   DBB   22744
12    DT   28497
13   DTS   11382
14   PBB     880
15   PBS     882
16  SBRT   16388
17   SSD   87128
18  SSRT    6777
19   SSU   86691
20   TAG   14475
21   TAR   14117


### Load Lookback Periods
- This LOOKBACK_DF is what I use to re-test the previous 12-month period every 1-month
  1. So every month I look at the previous 12-months starting on the last day of the previous month
  2. I want to create new models from the previous 12-month period
  3. I will backtest/validate the models by running the current month data through the model

In [13]:
### Load Lookback Periods
#######################################
LOOKBACK_DF = Periods.lookback_df(LB_MONTHS=12,NEXT=False,MIN_DATE=None,MAX=None,LB_MIN_DATE=None)
LOOKBACK_DF = LOOKBACK_DF[['month_id','lookback_start','lookback_end']][(LOOKBACK_DF["month_id"] <= '2022_11')].head(12).reset_index(drop=True)
print(LOOKBACK_DF)

   month_id lookback_start lookback_end
0   2022_11     2021-11-01   2022-10-31
1   2022_10     2021-10-01   2022-09-30
2   2022_09     2021-09-01   2022-08-31
3   2022_08     2021-08-01   2022-07-31
4   2022_07     2021-07-01   2022-06-30
5   2022_06     2021-06-01   2022-05-31
6   2022_05     2021-05-01   2022-04-30
7   2022_04     2021-04-01   2022-03-31
8   2022_03     2021-03-01   2022-02-28
9   2022_02     2021-02-01   2022-01-31
10  2022_01     2021-01-01   2021-12-31
11  2021_12     2020-12-01   2021-11-30


### Loop through Lookback Periods
- Below is an example of what I would like to be able to do in creating & testing models for each lookback period


In [15]:
lst = []
for index, row in LOOKBACK_DF.iterrows():
    
    MONTH_ID = row["month_id"]
    lookback_start = pd.to_datetime(row["lookback_start"]).strftime('%Y-%m-%d')
    lookback_end = pd.to_datetime(row["lookback_end"]).strftime('%Y-%m-%d')
    LAST_MO_ID = pd.to_datetime(row["lookback_end"]).strftime('%Y_%m')
    print(f"\n{'='*100}\nRun for {MONTH_ID} \t Lookback Range {lookback_start} to {lookback_end}\n{'='*100}\n")
    
    for PCODE in ALL_CODES:        
        ### Load Production Data (For Backtesting)
        #######################################
        model_production_data = Drct.load_pcode(AGG=AGG,PCODE=PCODE,PP=True,LAST_MO_ID=MONTH_ID,MONTHS_BACK=1,PRINT=True)
        ### Model Creation Data
        #######################################
        model_creation_data = Drct.load_pcode(AGG=AGG,PCODE=PCODE,PP=True,LAST_MO_ID=LAST_MO_ID,MONTHS_BACK=12,PRINT=True)
        
        ### Define Column Types
        #######################################
        cols = model_creation_data.columns.tolist()
        basic_is = lambda x: True in [ i in x for i in ['_ftN_',]]
        category_is = lambda x: True in [ i in x for i in ['_ftC_',]]
        utility_is = lambda x: True in [ i in x for i in ['_UTILITY_',]]
        cols_attributes_basic = [ i for i in cols if basic_is(i) ]
        cols_attributes_category = [ i for i in cols if category_is(i) ]
        cols_utility = [ i for i in cols if utility_is(i) ]
        column_target = '_TARGET'
        
        columns_dist = {'cols_attributes_basic': cols_attributes_basic,
                'cols_attributes_category': cols_attributes_category,
                'column_target': column_target}
        
        
        ## RUN :: Model Creation
        ######################################
#         create_model(model_creation_data,AGG,PCODE,MONTH_ID, columns_dist)
        
        ### RUN :: Model Production Predict
        #######################################
        model_production_data, stats = run_production_predict(model_production_data,AGG,PCODE,MONTH_ID, columns_dist)
        lst.append(stats)



Run for 2022_11 	 Lookback Range 2021-11-01 to 2022-10-31

3BPD 	 Pre-Processed 		 1 Months 	 2022-11-01 to 2022-11-30 	 Len: 673 	 Cols: 212
3BPD 	 Pre-Processed 		 12 Months 	 2021-11-01 to 2022-10-31 	 Len: 6,066 	 Cols: 212

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR '3BPD' | 2022_11
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

3BPU 	 Pre-Processed 		 1 Months 	 2022-11-01 to 2022-11-30 	 Len: 663 	 Cols: 212
3BPU 	 Pre-Processed 		 12 Months 	 2021-11-01 to 2022-10-31 	 Len: 6,073 	 Cols: 212

	---------------------------------------------

DBB 	 Pre-Processed 		 12 Months 	 2021-11-01 to 2022-10-31 	 Len: 8,310 	 Cols: 206

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'DBB' | 2022_11
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

DT 	 Pre-Processed 		 1 Months 	 2022-11-01 to 2022-11-30 	 Len: 898 	 Cols: 210
DT 	 Pre-Processed 		 12 Months 	 2021-11-01 to 2022-10-31 	 Len: 9,283 	 Cols: 210

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'DT' | 2022_11
	------------------------------------------------------------
	1. Load Saved Model Files c

3BPD 	 Pre-Processed 		 12 Months 	 2021-10-01 to 2022-09-30 	 Len: 5,718 	 Cols: 212

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR '3BPD' | 2022_10
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

3BPU 	 Pre-Processed 		 1 Months 	 2022-10-03 to 2022-10-31 	 Len: 779 	 Cols: 212
3BPU 	 Pre-Processed 		 12 Months 	 2021-10-01 to 2022-09-30 	 Len: 5,532 	 Cols: 212

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR '3BPU' | 2022_10
	------------------------------------------------------------
	1. Load Saved Model

	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

DT 	 Pre-Processed 		 1 Months 	 2022-10-03 to 2022-10-31 	 Len: 990 	 Cols: 210
DT 	 Pre-Processed 		 12 Months 	 2021-10-01 to 2022-09-30 	 Len: 8,633 	 Cols: 210

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'DT' | 2022_10
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	---------------------

	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

3BPU 	 Pre-Processed 		 1 Months 	 2022-09-01 to 2022-09-30 	 Len: 464 	 Cols: 212
3BPU 	 Pre-Processed 		 12 Months 	 2021-09-01 to 2022-08-31 	 Len: 5,293 	 Cols: 212

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR '3BPU' | 2022_09
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	---------------

...

SBRT 	 Pre-Processed 		 1 Months 	 2022-02-01 to 2022-02-28 	 Len: 550 	 Cols: 210
SBRT 	 Pre-Processed 		 12 Months 	 2021-02-01 to 2022-01-28 	 Len: 3,563 	 Cols: 210

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'SBRT' | 2022_02
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

SSD 	 Pre-Processed 		 1 Months 	 2022-02-01 to 2022-02-28 	 Len: 1,953 	 Cols: 206
SSD 	 Pre-Processed 		 12 Months 	 2021-02-01 to 2022-01-31 	 Len: 22,090 	 Cols: 206

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'SSD' | 2022_02

BOU 	 Pre-Processed 		 1 Months 	 2022-01-03 to 2022-01-31 	 Len: 992 	 Cols: 210
BOU 	 Pre-Processed 		 12 Months 	 2021-01-04 to 2021-12-31 	 Len: 17,102 	 Cols: 210

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'BOU' | 2022_01
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

BTRD 	 Pre-Processed 		 1 Months 	 2022-01-03 to 2022-01-28 	 Len: 72 	 Cols: 213
BTRD 	 Pre-Processed 		 12 Months 	 2021-01-04 to 2021-12-31 	 Len: 633 	 Cols: 213

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'BTRD' | 2022_01
	---

	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

SSD 	 Pre-Processed 		 1 Months 	 2022-01-03 to 2022-01-31 	 Len: 1,685 	 Cols: 206
SSD 	 Pre-Processed 		 12 Months 	 2021-01-04 to 2021-12-31 	 Len: 23,245 	 Cols: 206

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'SSD' | 2022_01
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	---------------

	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

BTRD 	 Pre-Processed 		 1 Months 	 2021-12-01 to 2021-12-31 	 Len: 37 	 Cols: 213
BTRD 	 Pre-Processed 		 12 Months 	 2020-12-01 to 2021-11-30 	 Len: 660 	 Cols: 213

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'BTRD' | 2021_12
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------

SSD 	 Pre-Processed 		 1 Months 	 2021-12-01 to 2021-12-31 	 Len: 1,808 	 Cols: 206
SSD 	 Pre-Processed 		 12 Months 	 2020-12-01 to 2021-11-30 	 Len: 23,620 	 Cols: 206

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'SSD' | 2021_12
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------

SSRT 	 Pre-Processed 		 1 Months 	 2021-12-01 to 2021-12-30 	 Len: 153 	 Cols: 210
SSRT 	 Pre-Processed 		 12 Months 	 2020-12-01 to 2021-11-30 	 Len: 1,640 	 Cols: 210

	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR 'SSRT' | 2021_12

In [16]:
pd.set_option('display.max_rows', 1000)
df_stats = pd.concat(lst, axis=0)
df_stats[df_stats['pcode']=='3BPU']

,pcode,month_id,prediction_mean_squared_error,prediction_pearson_correlation,prediction_seconds
0,3BPU,2022_11,0.000085,0.117486,1.854157
0,3BPU,2022_10,0.000037,0.143467,2.116983
0,3BPU,2022_09,0.000032,-0.039384,2.353577
0,3BPU,2022_08,0.000026,0.094951,1.439488
0,3BPU,2022_07,0.000030,0.074545,2.173482
0,3BPU,2022_06,0.000049,0.093702,1.570165
0,3BPU,2022_05,0.000042,0.003003,1.570245
0,3BPU,2022_04,0.000032,0.121010,1.442842
0,3BPU,2022_03,0.000040,0.056699,0.942312
0,3BPU,2022_02,0.000043,-0.039127,1.633842


### NOTES About Column Naming Structure
- There are 3 primary types of columns:
    1. The target column (y) for creating/validating predictions. This column is named '_TARGET'
    2. Utility columns that is just here to help me inspect the data but these should not be used in creating predictions (starts with '_UTILITY_')
    2. Basic Attribute columns (X) that are numerical (but not neccesarily all linier) used for creating predictions (starts with '_ftN_')
    2. Categorical Attribute columns (X) that have already been converted from different strings to intigers used for creating predictions (starts with '_ftC_')


In [12]:
print(f"\nUtility Column Examples \t {cols_utility[0:5]}")
print(f"Basic Attributes Examples \t {cols_attributes_basic[0:5]}")
print(f"Category Attributes Examples \t {cols_attributes_category[0:5]}")
print(f"Target Column Is \t\t {column_target}\n")



Utility Column Examples 	 ['_UTILITY_date', '_UTILITY_puid']
Basic Attributes Examples 	 ['_ftN_open', '_ftN_high', '_ftN_low', '_ftN_close', '_ftN_volume']
Category Attributes Examples 	 ['_ftC_gap_type', '_ftC_gap_level', '_ftC_gap_code']
Target Column Is 		 _TARGET



### NOTES About model_production_data & model_creation_data
- The function Drct.load_pcode() Loads the model data for a specific date range.
- Notice the date range for the 'model_production_data' is a 1-month period
- Notice the date range for the 'model_creation_data' is the previous 12-month period


## Model Creation Master Function
- This will obviously but a much longer and more involved function but the main takeways are:
   - Even though we need to re-create these models, I think it's save to build off of your analysis that Random Forest & All Features seems to work best, so let's limit the model creation to that general scope for now.
   - It should save the model information for the specific lookback period into that period's folder
   - It should save whatever information is neccesary (i.e. pickle file) so that in production mode it can make as fast of a prediction as possible.

In [ ]:
### RUN :: Model Creation
#######################################
# create_model(model_creation_data,AGG,PCODE,MONTH_ID, columns_dist, Scaling)

## Model Production Master Function
- This function is meant to be a "mock version" of what I will end up using with live real-time data.
- In live production mode, I will take real-time data (just like 'model_production_data' except only a few rows at a time and not a whole month) and send it to this function in order to get a new 'predict' column in return.
- For backtesting purposes I would also like to have a 'stats' dataframe returned as well with run-time and accuracy scores.


In [13]:
### RUN :: Model Production Predict
#######################################
model_production_data, stats = run_production_predict(model_production_data,AGG,PCODE,MONTH_ID, columns_dist)


	------------------------------------------------------------
	RUN PRODUCTION PREDICT FOR '3BPD' | 2022_11
	------------------------------------------------------------
	1. Load Saved Model Files created from 'model_creation_data'
	2. Create Prediction Column in 'model_production_data'
	3. Create Stats KPI CSV File with 'model_production_data' results
	4. Save the updated 'model_production_data' and 'stats' to CSV
	5. Return the updated 'model_production_data' and 'stats' from function
	------------------------------------------------------------



In [16]:
pd.set_option('display.max_rows', 1000)

model_production_data

,_TARGET,predict,_UTILITY_date,_UTILITY_puid,_ftN_open,_ftN_high,_ftN_low,_ftN_close,_ftN_volume,_ftN_vwap,...,_ftN_vol_proj_spy_rel,_ftN_b1_breaks_sr,_ftN_b1_cons,_ftN_b1_pct_body,_ftN_b1_vol,_ftN_b1_wr,_ftN_from_datr,_ftN_inside_pct,_ftN_inside_vol,_ftN_vol_proj
0,0.9952,1.000288,2022-11-01,3BPD_202211010905_5T_CMA,1.000426,1.002841,0.997167,0.999574,0.501096,0.004342,...,0.0117,0.0,1.0,0.952,6.174894e-05,2.894,0.0,0.452,3.087447e-05,0.616194
1,0.9927,1.000546,2022-11-01,3BPD_202211010905_5T_CNK,1.000000,1.005060,0.994966,1.000000,0.497554,0.000364,...,0.0186,0.0,1.0,0.727,1.548391e-05,2.253,0.0,0.500,1.665440e-05,0.464599
2,0.9926,0.999249,2022-11-01,3BPD_202211010905_5T_JD,1.001279,1.005253,0.994774,0.998723,1.456212,0.000115,...,0.2418,0.0,1.0,0.710,6.695840e-06,4.663,0.0,0.306,4.264717e-06,2.057846
3,1.0027,0.999824,2022-11-01,3BPD_202211010905_5T_LEVI,1.001323,1.003307,0.996704,0.998679,0.261223,0.001593,...,0.0095,0.0,1.0,1.000,4.041254e-05,3.372,0.0,0.290,2.746790e-05,0.316608
4,0.9948,1.000522,2022-11-01,3BPD_202211010905_5T_MRO,0.998700,1.003256,0.996754,1.001301,0.640208,0.000259,...,0.1202,0.0,1.0,0.813,1.188721e-05,2.315,0.0,0.474,5.391789e-06,0.903292
5,1.0019,1.000586,2022-11-01,3BPD_202211010905_5T_NCR,0.999069,1.003262,0.996749,1.000932,0.911093,0.001349,...,0.0089,0.0,1.0,0.857,1.355730e-04,3.442,0.0,0.400,5.720565e-05,0.506735
6,1.0023,1.000547,2022-11-01,3BPD_202211010905_5T_PK,1.000765,1.004589,0.995432,0.999236,0.852182,0.000228,...,0.0260,0.0,2.0,0.880,2.234744e-05,3.223,0.0,0.480,1.481713e-05,0.765574
7,1.0028,1.000579,2022-11-01,3BPD_202211010905_5T_PLAY,0.996706,1.004323,0.995695,1.003305,0.291384,0.007716,...,0.0054,0.0,2.0,0.960,2.150685e-04,4.516,0.0,0.274,5.694716e-05,0.563533
8,1.0031,1.001437,2022-11-01,3BPD_202211010905_5T_SM,0.994758,1.006043,0.993994,1.005270,0.550603,0.002855,...,0.0109,0.0,1.0,1.000,4.110360e-05,2.537,0.0,0.589,3.447197e-05,0.581489
9,1.0019,1.000213,2022-11-01,3BPD_202211010905_5T_STWD,1.000000,1.001929,0.998075,1.000000,0.455451,0.001371,...,0.0203,0.0,1.0,0.741,8.434053e-05,2.254,0.0,0.506,3.005086e-05,0.555209


In [15]:
stats

,pcode,month_id,prediction_mean_squared_error,prediction_pearson_correlation,prediction_seconds
0,3BPD,2022_11,0.000054,0.066198,0.235314
